In [1]:
%%capture
!git clone https://github.com/nyu-mll/jiant.git
!pip install -r jiant/requirements-no-torch.txt

In [2]:
import sys
sys.path.insert(0, "/content/jiant")
import os
import jiant.utils.python.io as py_io
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache
import jiant.proj.main.runscript as main_runscript
import jiant.proj.main.export_model as export_model
import jiant.proj.main.scripts.configurator as configurator
import jiant.scripts.download_data.runscript as downloader
import jiant.shared.caching as caching
import jiant.utils.display as display

In [3]:
%%capture
%cd /content
# Download wic
!PYTHONPATH=/content/jiant python jiant/jiant/scripts/download_data/runscript.py \
    download \
    --tasks wic \
    --output_path=/content/tasks/

In [4]:
export_model.export_model(
    hf_pretrained_model_name_or_path="roberta-large",
    output_base_path="./models/roberta-large",
)

In [5]:
# Tokenize and cache each task
task_name = "wic"

tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
    task_config_path=f"./tasks/configs/{task_name}_config.json",
    hf_pretrained_model_name_or_path="roberta-large",
    output_dir=f"./cache/{task_name}",
    phases=["train", "val"],
))

WiCTask
  [train]: /content/tasks/data/wic/train.jsonl
  [test]: /content/tasks/data/wic/test.jsonl
  [val]: /content/tasks/data/wic/val.jsonl


Streaming output truncated to the last 5000 lines.
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
Mode

ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
ModelArchitectures.ROBERTA
M

In [6]:
jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
    task_config_base_path="./tasks/configs",
    task_cache_base_path="./cache",
    train_task_name_list=["wic"],
    val_task_name_list=["wic"],
    train_batch_size=16,
    eval_batch_size=16,
    epochs=6,
    num_gpus=2,
).create_config()
os.makedirs("./run_configs/", exist_ok=True)
py_io.write_json(jiant_run_config, "./run_configs/wic_run_config.json")
display.show_json(jiant_run_config)

{
  "task_config_path_dict": {
    "wic": "./tasks/configs/wic_config.json"
  },
  "task_cache_config_dict": {
    "wic": {
      "train": "./cache/wic/train",
      "val": "./cache/wic/val",
      "val_labels": "./cache/wic/val_labels"
    }
  },
  "sampler_config": {
    "sampler_type": "ProportionalMultiTaskSampler"
  },
  "global_train_config": {
    "max_steps": 850,
    "warmup_steps": 85
  },
  "task_specific_configs_dict": {
    "wic": {
      "train_batch_size": 16,
      "eval_batch_size": 16,
      "gradient_accumulation_steps": 1,
      "eval_subset_num": 500
    }
  },
  "taskmodels_config": {
    "task_to_taskmodel_map": {
      "wic": "wic"
    },
    "taskmodel_config_map": {
      "wic": null
    }
  },
  "task_run_config": {
    "train_task_list": [
      "wic"
    ],
    "train_val_task_list": [
      "wic"
    ],
    "val_task_list": [
      "wic"
    ],
    "test_task_list": []
  },
  "metric_aggregator_config": {
    "metric_aggregator_type": "EqualMetricAggregato

In [7]:
run_args = main_runscript.RunConfiguration(
    jiant_task_container_config_path="./run_configs/wic_run_config.json",
    output_dir="./runs/wic",
    hf_pretrained_model_name_or_path="roberta-large",
    model_path="./models/roberta-large/model/model.p",
    model_config_path="./models/roberta-large/model/config.json",
    learning_rate = 1e-5,
    eval_every_steps=500,
    do_train=True,
    do_val=True,
    do_save=True,
    force_overwrite=True,

)
main_runscript.run_loop(run_args)

  jiant_task_container_config_path: ./run_configs/wic_run_config.json
  output_dir: ./runs/wic
  hf_pretrained_model_name_or_path: roberta-large
  model_path: ./models/roberta-large/model/model.p
  model_config_path: ./models/roberta-large/model/config.json
  model_load_mode: from_transformers
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 500
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: True
  seed: -1
  learning_rate: 1e-05
  adam_epsilon: 1e-08
  max_grad_norm: 1.0
  optimizer_type: adam
  no_cuda: False
  fp16: False
  fp16_opt_level: O1
  local_rank: -1
  server_ip: 
  server_port: 
device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
Using seed: 1109065334
{
  "jiant_task_container_config_path": "./run_configs/wic_run_config.json",
  "output_dir": 

/content/jiant/jiant/proj/main/modeling/model_setup.py:174: UserWarning: The following weights were not loaded: dict_keys(['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias'])
  "The following weights were not loaded: {}".format(remainder_weights_dict.keys())


No optimizer decay for:
  encoder.embeddings.LayerNorm.weight
  encoder.embeddings.LayerNorm.bias
  encoder.encoder.layer.0.attention.self.query.bias
  encoder.encoder.layer.0.attention.self.key.bias
  encoder.encoder.layer.0.attention.self.value.bias
  encoder.encoder.layer.0.attention.output.dense.bias
  encoder.encoder.layer.0.attention.output.LayerNorm.weight
  encoder.encoder.layer.0.attention.output.LayerNorm.bias
  encoder.encoder.layer.0.intermediate.dense.bias
  encoder.encoder.layer.0.output.dense.bias
  encoder.encoder.layer.0.output.LayerNorm.weight
  encoder.encoder.layer.0.output.LayerNorm.bias
  encoder.encoder.layer.1.attention.self.query.bias
  encoder.encoder.layer.1.attention.self.key.bias
  encoder.encoder.layer.1.attention.self.value.bias
  encoder.encoder.layer.1.attention.output.dense.bias
  encoder.encoder.layer.1.attention.output.LayerNorm.weight
  encoder.encoder.layer.1.attention.output.LayerNorm.bias
  encoder.encoder.layer.1.intermediate.dense.bias
  encode


Loading Best



{
  "aggregated": 0.7100313479623824,
  "wic": {
    "loss": 0.7485662378370762,
    "metrics": {
      "major": 0.7100313479623824,
      "minor": {
        "acc": 0.7100313479623824
      }
    }
  }
}
